In [20]:
# Treino do modelo com hiperparâmetros selecionados
import torch
import gymnasium
from gymnasium.wrappers.record_video import RecordVideo
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from pushbullet import Pushbullet

pb = Pushbullet("o.8HIa4sqhWdBgCcIm9E5TQ34Y07EEcFHN")
torch.manual_seed(1)

ENV_ID = 'BipedalWalker-v3'
N_ENVS = 1
HM = False
SIZE = [128,256]
TIMESTEPS = int(2e6)

train_env = make_vec_env(ENV_ID, n_envs= N_ENVS, seed=0, wrapper_class=Monitor, env_kwargs= dict(hardcore = HM))
best_model = PPO('MlpPolicy', train_env,  policy_kwargs= dict(net_arch = dict(pi=SIZE, vf=SIZE)) )

seeds = [1, 2, 3, 5, 8]
for seed in seeds:
    best_model.set_random_seed(seed)
    best_model.learn(total_timesteps=TIMESTEPS)

env = gymnasium.make(ENV_ID, render_mode = 'rgb_array', hardcore = HM)
env = RecordVideo(env, video_folder= ENV_ID, name_prefix= str(HM) + str(N_ENVS) + str(SIZE) + str(TIMESTEPS), episode_trigger= lambda x: True, disable_logger = True)
obs = env.reset()[0]
best_model.set_random_seed(144)
done = False
while not done:
    action, _ = best_model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
env.close()

pb.push_note("Fim da execução!", "Seu código no computador do laboratório terminou de rodar.")
print('Done.')

Done.


In [11]:
# Visualização
import gymnasium
from stable_baselines3.ppo import PPO
from gymnasium.wrappers.record_episode_statistics import RecordEpisodeStatistics
from gymnasium.wrappers.record_video import RecordVideo
import keyboard

def verificar(best_model: PPO):
    env = gymnasium.make(ENV_ID, render_mode = 'rgb_array', hardcore = HM)
    obs = env.reset()[0]
    i = 0
    seeds = [13, 21, 34, 55, 89]
    while i < 5:
        best_model.set_random_seed(seeds[i])
        action, _ = best_model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        if terminated or truncated:
            obs, info = env.reset()
            i = i + 1
        if keyboard.is_pressed('esc'):
            break
    env.close()
    
verificar(best_model)